In [1]:
! /opt/bin/nvidia-smi

Mon Dec  7 04:11:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content') 

Mounted at /content/drive


In [3]:
import shutil
DIR = '/content/drive/MyDrive/Colab Notebooks/HW3'
shutil.copyfile(os.path.join(DIR, 'HW3.zip'), 'HW3.zip')
!unzip HW3.zip

Archive:  HW3.zip
   creating: HW3/
  inflating: HW3/.DS_Store           
   creating: __MACOSX/
   creating: __MACOSX/HW3/
  inflating: __MACOSX/HW3/._.DS_Store  
   creating: HW3/.ipynb_checkpoints/
  inflating: HW3/.ipynb_checkpoints/HW3-checkpoint.ipynb  
   creating: HW3/__pycache__/
  inflating: HW3/__pycache__/utils.cpython-37.pyc  
   creating: HW3/dataset/
  inflating: HW3/dataset/.DS_Store   
   creating: __MACOSX/HW3/dataset/
  inflating: __MACOSX/HW3/dataset/._.DS_Store  
   creating: HW3/dataset/test/
  inflating: HW3/dataset/test/2007_000629.jpg  
   creating: __MACOSX/HW3/dataset/test/
  inflating: __MACOSX/HW3/dataset/test/._2007_000629.jpg  
  inflating: HW3/dataset/test/2007_001175.jpg  
  inflating: __MACOSX/HW3/dataset/test/._2007_001175.jpg  
  inflating: HW3/dataset/test/2007_001239.jpg  
  inflating: __MACOSX/HW3/dataset/test/._2007_001239.jpg  
  inflating: HW3/dataset/test/2007_001733.jpg  
  inflating: __MACOSX/HW3/dataset/test/._2007_001733.jpg  
  inflating:

In [12]:
os.chdir('/content/HW3')

from PixelLib.pixellib.custom_train import instance_custom_training
from utils import get_imagenet_weights
import glob

class_names = ['BG', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
save_models_path = os.path.join(DIR, 'mask_rcnn_models512_12/7')

def get_bestModel_weights():
  return os.path.join(os.path.abspath(save_models_path), sorted(os.listdir(save_models_path), reverse=True)[0])
   
#params
init_lr = 0.005
lr_steps = 20
max_dim = 512
min_dim = 512


In [5]:
#infer
def infer():
  train_maskrcnn = instance_custom_training()
  train_maskrcnn.load_dataset("dataset")
  train_maskrcnn.modelConfig(network_backbone="resnet101", num_classes=20, class_names=class_names, batch_size=1, detection_threshold=0.5, image_max_dim=max_dim, image_min_dim=min_dim)
  #train_maskrcnn.evaluate_model(model_path=save_models_path)
  train_maskrcnn.evaluate_model(model_path=get_bestModel_weights())


In [ ]:
#train
train_maskrcnn = instance_custom_training()
train_maskrcnn.load_dataset("dataset")


#1st
train_maskrcnn.modelConfig(network_backbone="resnet101", num_classes=20, class_names=class_names, batch_size=8, detection_threshold=0.7, image_max_dim=max_dim, image_min_dim=min_dim, learning_rate=init_lr, lr_steps=lr_steps)
train_maskrcnn.load_pretrained_model(get_imagenet_weights())
train_maskrcnn.train_model(num_epochs=30, augmentation=True, layers='heads', path_trained_models=save_models_path)
infer()
#2nd
train_maskrcnn.modelConfig(network_backbone="resnet101", num_classes=20, class_names=class_names, batch_size=8, detection_threshold=0.7, image_max_dim=max_dim, image_min_dim=min_dim, learning_rate=init_lr, lr_steps=lr_steps)
train_maskrcnn.load_pretrained_model(get_bestModel_weights())
train_maskrcnn.train_model(num_epochs=60, augmentation=True, layers='4+', path_trained_models=save_models_path)
infer()

#3st
train_maskrcnn.modelConfig(network_backbone="resnet101", num_classes=20, class_names=class_names, batch_size=4, detection_threshold=0.7, image_max_dim=max_dim, image_min_dim=min_dim, learning_rate=init_lr/10, lr_steps=lr_steps)
train_maskrcnn.load_pretrained_model(get_bestModel_weights())
train_maskrcnn.train_model(num_epochs=200, augmentation=True, layers='all', path_trained_models=save_models_path)
infer()
'''
#3st
train_maskrcnn.modelConfig(network_backbone="resnet101", num_classes=20, class_names=class_names, batch_size=8, detection_threshold=0.5, image_max_dim=max_dim, image_min_dim=min_dim, learning_rate=init_lr, lr_steps=lr_steps)
train_maskrcnn.load_pretrained_model(get_imagenet_weights())
train_maskrcnn.train_model(num_epochs=30, augmentation=True, layers='all', path_trained_models=save_models_path)
'''

Using resnet101 as network backbone For Mask R-CNN model
Train 1349 images
Validate 1349 images
Applying augmentation on dataset
Checkpoint Path: /content/drive/MyDrive/Colab Notebooks/HW3/mask_rcnn_models512
Selecting layers to train
Learning Rate 0.0005
Learning Rate 0.0005
Epoch 1/200
180/180 [==============================] - 428s 2s/step - batch: 89.5000 - size: 4.0000 - loss: 3.9791 - rpn_class_loss: 0.1886 - rpn_bbox_loss: 1.8560 - mrcnn_class_loss: 0.6554 - mrcnn_bbox_loss: 0.6635 - mrcnn_mask_loss: 0.6157
Learning Rate 0.0005
Epoch 2/200
180/180 [==============================] - 271s 2s/step - batch: 89.5000 - size: 4.0000 - loss: 3.8903 - rpn_class_loss: 0.1819 - rpn_bbox_loss: 2.1730 - mrcnn_class_loss: 0.4835 - mrcnn_bbox_loss: 0.5549 - mrcnn_mask_loss: 0.4970
Learning Rate 0.0005
Epoch 3/200
180/180 [==============================] - 290s 2s/step - batch: 89.5000 - size: 4.0000 - loss: 3.8027 - rpn_class_loss: 0.1602 - rpn_bbox_loss: 2.2185 - mrcnn_class_loss: 0.4313 - mr

In [6]:
infer()

Using resnet101 as network backbone For Mask R-CNN model
Instructions for updating:
Use fn_output_signature instead
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
/content/drive/MyDrive/Colab Notebooks/HW3/mask_rcnn_models512/mask_rcnn_model.021-2.737345.h5 evaluation using iou_threshold 0.5 is 0.161006 



In [7]:
import numpy as np
from itertools import groupby
from pycocotools import mask as maskutil

def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    compressed_rle = maskutil.frPyObjects(rle, rle.get('size')[0], rle.get('size')[1])
    compressed_rle['counts'] = str(compressed_rle['counts'], encoding='utf-8')
    return compressed_rle

In [8]:
from PixelLib.pixellib.instance import custom_segmentation 
from utils import binary_mask_to_rle
import json, os
import numpy as np

json_path = '/content/HW3/dataset/test.json'
with open(json_path) as f:
   test_json = json.load(f)

test_path = '/content/HW3/dataset/test'
save_dir = os.path.join(DIR, 'test_predict')
if not os.path.exists(save_dir):
  os.mkdir(save_dir)

submissions = []

segment_image = custom_segmentation()
segment_image.inferConfig(num_classes=20, class_names=class_names, image_max_dim=max_dim, image_min_dim=min_dim, detection_threshold=0.5)
segment_image.load_model(get_bestModel_weights())

for t in test_json['images']:
  filename = t['file_name']
  id = t['id']
  filepath = os.path.join(test_path, filename)
  save_path = os.path.join(save_dir, filename)
  results, o = segment_image.segmentImage(filepath, show_bboxes=True, output_image_name=save_path)
  #print(results)
  category_ids = results['class_ids']
  scores = results['scores']
  segmentations = results['masks']
  n_instances = len(scores)    
  if len(category_ids) > 0: # If any objects are detected in this image
    for i in range(n_instances):
      instance = dict()
      category_id, score, segmentation = category_ids[i], scores[i], binary_mask_to_rle(segmentations[:,:,i])
      #print(segmentations[:,:,i])
      instance['image_id'], instance['category_id'], instance['segmentation'], instance['score'] = \
        int(id), int(category_id), segmentation, float(score)
      submissions.append(instance)
      #print(segmentation)

#submissions = tuple(submissions)
with open(os.path.join(DIR,"submission.json"), "w") as f:
    json.dump(submissions, f)

Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image save

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
 
# Root directory of the project
ROOT_DIR = os.path.abspath("../")
 
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.config import Config
 
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
 
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_cells_0001.h5")
 
# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
 
class InferenceConfig(Config):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    """Configuration for training on MS COCO.
    Derives from the base Config class and overrides values specific
    to the COCO dataset.
    """
    # Give the configuration a recognizable name
    NAME = "cells"
 
    RPN_ANCHOR_SCALES = (8 * 2, 16 * 2, 32 * 2, 64 * 2, 128 * 2)
 
    IMAGE_MIN_DIM = min_dim
    IMAGE_MAX_DIM = max_dim
 
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1
 
    # Uncomment to train on 8 GPUs (default is 1)
    GPU_COUNT = 1
 
    # Number of classes (including background)
    NUM_CLASSES = 1 + 20
 
config = InferenceConfig()
config.display()
 
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
 
# Load weights trained on MS-COCO
model.load_weights(get_bestModel_weights(), by_name=True)
class_names = ['BG', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

import json
import numpy as np

json_path = '/content/HW3/dataset/test.json'
with open(json_path) as f:
   test_json = json.load(f)

test_path = '/content/HW3/dataset/test'
save_dir = os.path.join(DIR, 'test_predict')
if not os.path.exists(save_dir):
  os.mkdir(save_dir)

submissions = []

# Load a random image from the images folder
#file_names = next(os.walk(IMAGE_DIR))[2]
#image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
for t in test_json['images']:
  filename = t['file_name']
  id = t['id']
  filepath = os.path.join(test_path, filename)
  save_path = os.path.join(save_dir, filename)
  image=io.imread(filepath)
  results = model.detect([image], verbose=1)
  rresults = results[0]
  #print(results)
  category_ids = results['class_ids']
  scores = results['scores']
  segmentations = results['masks']
  n_instances = len(score)    
  if len(categories) > 0: # If any objects are detected in this image
    for i in range(n_instances):
      instance = dict()
      category_id, score, segmentation = category_ids[i], scores[i], binary_mask_to_rle(segmentations[:,:,i])
      print(segmentations[i].shape)
      instance['image_id'], instance['score'], instance['category_id'], instance['segmentation'] = \
        int(id), float(score), int(category_id), str(segmentation)
      submissions.append(instance)
      print(segmentation)
